# Aircraft Data

## Introduction
Aircraft data was gathered by checking all occurring tail numbers in the flights data table by web scraping the following site. __[https://www.airfleets.net/recherche?key='tail_number'](https://www.airfleets.net/recherche/)__
With that I could combine my flight data with the aircrafts which are included in the flights data. Some values were still missing and had to be checked by hand.

Sheet für ADG group und wingspan:
__[https://skybrary.aero/articles/airplane-design-group-adg](https://skybrary.aero/articles/airplane-design-group-adg)__
Sheet für AAC:
__[https://en.wikipedia.org/wiki/Aircraft_approach_category](https://en.wikipedia.org/wiki/Aircraft_approach_category)__
Sheet für Wake Turbolence Category:
__[https://skybrary.aero/articles/icao-wake-turbulence-category](https://skybrary.aero/articles/icao-wake-turbulence-category)__



In [1]:
import pandas as pd
import os
from main.utils.data_manage_utils import read_pickle_from_subfolder

# Variables
ROOT_PATH = os.path.abspath("../../")
INPUT_FOLDER = os.path.join(ROOT_PATH, "data/input")
OUTPUT_FOLDER = os.path.join(ROOT_PATH, "data/preparation/prepped_files")

In [2]:
#Loading aircrafts data
aircrafts_df = pd.read_csv(os.path.join(INPUT_FOLDER,"data_raw/all_aircrafts_FAA.csv"),encoding='latin-1', on_bad_lines='warn', sep=";")
print("="*20 + "AIRCRAFTS DESCRIBE" + "="*20)
print(aircrafts_df.describe())
print("="*20 + "AIRCRAFTS PRINT" + "="*20)
print(aircrafts_df.head())
print("=" * 20 + "AIRCRAFTS INFO" + "=" * 20)
aircrafts_df.info()

#Loading prepared flight data from pickle folder
atl_flights = pd.read_pickle(os.path.join(OUTPUT_FOLDER, "01_flights.pkl"))

====================AIRCRAFTS DESCRIBE====================
       Date Completed Manufacturer   Model Physical Class (Engine) Engines  \
count             873         2764    2764                    2764    2764   
unique             72          424    2559                       8       6   
top               tbd       Boeing  BK-117                     tbd     tbd   
freq              163          113       5                    1831    1956   

             AAC       ADG   TDG Approach Speed(Vref) Wingtip Configuration  \
count       2764      2764  2762                  853                  2764   
unique         6         8    10                  113                    21   
top     No Value  No Value   tbd                  140                   tbd   
freq        1911      1899  2036                   41                  2023   

        ... MGW(Outer to Outer)  MTOW Max RampMax Taxi Main Gear Config  \
count   ...                2764  2764             2764             2764   
uniq

In [3]:
all_planes_df = read_pickle_from_subfolder(os.path.join(INPUT_FOLDER,"scraped_aircraft/*.pkl"))
all_planes_df = all_planes_df.rename(columns = {'Regist.':'TAIL_NUM'})
drop_idx = all_planes_df.sort_values(by=["TAIL_NUM","Status"])[all_planes_df[["TAIL_NUM", "Status"]].duplicated(subset="TAIL_NUM", keep=False)].groupby("TAIL_NUM", group_keys=False).apply(lambda x: x.loc[x.Status != "Active"]).index
all_planes_df = all_planes_df.drop(labels= drop_idx, axis="index")
all_planes_df = all_planes_df[~all_planes_df.duplicated(subset="TAIL_NUM", keep="first")]

all_planes_df.loc[all_planes_df.loc[all_planes_df["Aircraft"] == "Canadair Regional Jet"].index,"Aircraft"] = "Bombardier Canadair Regional Jet"
all_planes_df

C:\Users\chrig\AppData\Local\Temp\ipykernel_10588\3949915742.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  drop_idx = all_planes_df.sort_values(by=["TAIL_NUM","Status"])[all_planes_df[["TAIL_NUM", "Status"]].duplicated(subset="TAIL_NUM", keep=False)].groupby("TAIL_NUM", group_keys=False).apply(lambda x: x.loc[x.Status != "Active"]).index


,Aircraft,TAIL_NUM,MSN,Airline,Status
0,Bombardier Canadair Regional Jet,N925EV,7831,SkyWest USAAtlantic Southeast AirlinesExpressjet,Active
1,Bombardier Canadair Regional Jet,N856AS,7404,SkyWest USAAtlantic Southeast AirlinesExpressjet,Active
2,Bombardier Canadair Regional Jet,N861AS,7445,SkyWest USAAtlantic Southeast AirlinesExpressjet,Stored
3,Bombardier Canadair Regional Jet,N858AS,7417,Atlantic Southeast AirlinesExpressjet,On order
4,Bombardier Canadair Regional Jet,N855AS,7395,Atlantic Southeast AirlinesExpressjetSkyWest USA,Stored
...,...,...,...,...,...
3323,Boeing 737,N624SW,27934,Southwest Airlines,Scrapped
3324,Boeing 737,N394SW,27380,Southwest Airlines,Stored
3325,Airbus A319,N342NB,1746,Delta Air LinesNorthwest Airlines,Active
3326,Boeing 737,N626SW,27702,iAero AirwaysSouthwest AirlinesSwift Air,Active


In [4]:
print(atl_flights.shape)
print(all_planes_df.shape)

(739429, 12)
(3267, 5)


In [5]:
all_planes_df.to_pickle(os.path.join(OUTPUT_FOLDER, "04_b_aircrafts.pkl"))

In [6]:
atl_flights_df = pd.merge(atl_flights,all_planes_df[["TAIL_NUM","Aircraft"]], how="inner", on="TAIL_NUM")

In [7]:
atl_flights_df["Aircraft"]

0         Airbus A320
1         Airbus A320
2         Airbus A320
3         Airbus A320
4         Airbus A320
             ...     
722560    Airbus A321
722561    Airbus A321
722562    Airbus A321
722563    Airbus A321
722564    Airbus A321
Name: Aircraft, Length: 722565, dtype: object

In [8]:
aircrafts_df[aircrafts_df["Manufacturer"] == "Boeing"]

,Date Completed,Manufacturer,Model,Physical Class (Engine),Engines,AAC,ADG,TDG,Approach Speed(Vref),Wingtip Configuration,...,MGW(Outer to Outer),MTOW,Max RampMax Taxi,Main Gear Config,ICAO Code,Wake Category,ATCT Weight Class,Years Manufactured,Note,"Parking Area (WS x Length), sf"
469,NaN,Boeing,720,tbd,tbd,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,B720,M,tbd,tbd,tbd,#WERT!
470,NaN,Boeing,314A Clipper,Piston,4,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,tbd,tbd,tbd,tbd,tbd,#WERT!
471,NaN,Boeing,707-020,tbd,tbd,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,tbd,M,tbd,tbd,tbd,#WERT!
472,NaN,Boeing,707-120B,Jet,4,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,B701,tbd,tbd,tbd,tbd,#WERT!
473,NaN,Boeing,707-320/420,Jet,4,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,B703,H,tbd,tbd,tbd,#WERT!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,NaN,Boeing,PT-18 Kaydet (Stearman),Piston,1,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,S,ST75,L,Small Eqpt,tbd,tbd,#WERT!
578,NaN,Boeing,PT-27 Kaydet (Stearman),Piston,1,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,S,ST75,L,Small Eqpt,tbd,tbd,#WERT!
579,NaN,Boeing,Raptor F22,tbd,tbd,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,tbd,tbd,tbd,tbd,tbd,#WERT!
580,NaN,Boeing,RC-135,tbd,tbd,No Value,No Value,tbd,NaN,tbd,...,tbd,tbd,tbd,tbd,R135,H,tbd,tbd,tbd,#WERT!


In [9]:
atl_flights_df["Aircraft"].unique()

array(['Airbus A320', 'Boeing 737 NG / Max', 'Boeing 757',
       'Bombardier Canadair Regional Jet', 'McDonnell Douglas MD-80/90',
       'Boeing 717', 'Airbus A319', 'Boeing 767', 'Airbus A321',
       'Boeing 737', 'Embraer 170/175', 'Embraer 190/195', 'Airbus A330',
       'Embraer 135/145', 'Boeing 777', 'Airbus A300', 'ATR 42/72',
       'Boeing 787', 'McDonnell Douglas DC-10', 'Boeing 747'],
      dtype=object)

In [10]:
pd.DataFrame(atl_flights_df["Aircraft"].unique())[0].str.extract('([a-zA-Z]+.*.+\d.*)', expand=True)

,0
0,Airbus A320
1,Boeing 737 NG / Max
2,Boeing 757
3,NaN
4,McDonnell Douglas MD-80/90
5,Boeing 717
6,Airbus A319
7,Boeing 767
8,Airbus A321
9,Boeing 737


In [11]:
#aircraft_split = atl_flights_df["Aircraft"].str.extract('([a-zA-Z]+.*.+\d.*)', expand=True)
aircraft_split = pd.DataFrame(atl_flights_df["Aircraft"].copy())
aircraft_split.rename(columns={"Aircraft" : "Model"}, inplace=True)
aircraft_split["Model"].unique()

array(['Airbus A320', 'Boeing 737 NG / Max', 'Boeing 757',
       'Bombardier Canadair Regional Jet', 'McDonnell Douglas MD-80/90',
       'Boeing 717', 'Airbus A319', 'Boeing 767', 'Airbus A321',
       'Boeing 737', 'Embraer 170/175', 'Embraer 190/195', 'Airbus A330',
       'Embraer 135/145', 'Boeing 777', 'Airbus A300', 'ATR 42/72',
       'Boeing 787', 'McDonnell Douglas DC-10', 'Boeing 747'],
      dtype=object)

In [12]:
aircraft_split.loc[aircraft_split["Model"] == "Canadair Regional Jet"] = "Bombardier Canadair Regional Jet"
aircraft_split["Model"].unique()

array(['Airbus A320', 'Boeing 737 NG / Max', 'Boeing 757',
       'Bombardier Canadair Regional Jet', 'McDonnell Douglas MD-80/90',
       'Boeing 717', 'Airbus A319', 'Boeing 767', 'Airbus A321',
       'Boeing 737', 'Embraer 170/175', 'Embraer 190/195', 'Airbus A330',
       'Embraer 135/145', 'Boeing 777', 'Airbus A300', 'ATR 42/72',
       'Boeing 787', 'McDonnell Douglas DC-10', 'Boeing 747'],
      dtype=object)

In [13]:
aircraft_split.shape

(722565, 1)

In [14]:
data = []
row = []
row_list = [255,143,151,154,159,164,475,487,500,514,527,532,542,549,683,1129,1099,1108,583, 1803]
name_list = ["Model_"]
for col_name in aircrafts_df.columns:
    name_list.append(col_name)
for count, entry in enumerate(aircraft_split["Model"].sort_values().unique()):
    row = []
    row.append(entry)
    for c, i in enumerate(aircrafts_df.loc[row_list[count]]):
        row.append(i)
    data.append(row)
aircrafts_atl_df = pd.DataFrame(data, columns=name_list)
aircrafts_atl_df = aircrafts_atl_df.drop(["Note","Years Manufactured","Manufacturer","Model", "Date Completed"], axis="columns")
aircrafts_atl_df.rename(columns={"Model_":"Model"}, inplace=True)
aircrafts_atl_df

,Model,Physical Class (Engine),Engines,AAC,ADG,TDG,Approach Speed(Vref),Wingtip Configuration,"Wingspan, ft","Length, ft",...,"Wheelbase, ft",Cockpit to Main Gear (CMG),MGW(Outer to Outer),MTOW,Max RampMax Taxi,Main Gear Config,ICAO Code,Wake Category,ATCT Weight Class,"Parking Area (WS x Length), sf"
0,ATR 42/72,Turboprop,2,B,III,1B or 2,114,no winglets,"88,75","89,13",...,"35,33","35,33","> 13' 5"" (16' per Appx 1)",49 603,49 978,D,AT72,M,tbd,7 910
1,Airbus A300,Jet,2,C,IV,5,137,wingtip fences,"147,14","177,43",...,"61,03","75,00","36,00",363 760,365 740,2D,A306,H,Heavy Eqpt,26 107
2,Airbus A319,Jet,2,C,III,3,126,wingtip fences,"111,88","111,02",...,"36,23","44,90","29,36",168 653,169 535,D,A319,M,Large Jet Eqpt,12 421
3,Airbus A320,Jet,2,C,III,3,136,wingtip fences,"111,88","123,27",...,"41,47","50,20","29,36",171 961,172 842,D,A320,M,Large Jet Eqpt,13 791
4,Airbus A321,Jet,2,C,III,3,140,wingtip fences,"111,88","146,03",...,"55,45","44,95","29,43",206 132,207 014,D,A321,M,Large Jet Eqpt,16 338
5,Airbus A330,Jet,2,C,V,5,137,winglets,"197,83","208,99",...,"83,27","97,28","41,37",533 519,535 503,2D,A333,H,Heavy Eqpt,41 344
6,Boeing 717,Jet,2,C,III,2,139,no winglets,"93,33","124,00",...,"57,75","57,75","19,43",118 000,119 000,D,B712,M,tbd,11 573
7,Boeing 737,Jet,2,C,III,3,129,no winglets,"93,00","100,17",...,"37,33","42,58","20,88",115 500,116 000,D,B732,M,Large Jet Eqpt,9 316
8,Boeing 737 NG / Max,Jet,2,D,III,3,141,no winglets,"112,58","138,17",...,"56,33","61,58","22,96",174 200,174 700,D,B739,M,Large Jet Eqpt,15 555
9,Boeing 747,Jet,4,D,V,5,157,winglets,"195,67","231,85",...,"84,00","91,67","41,33",610 000,613 500,2D/2D2,B744,H,tbd,45 366


In [15]:
aircrafts_atl_df.at[14,"Engines"] = 2
aircrafts_atl_df.at[14,"Wake Category"] = 'M'

aircrafts_atl_df.at[18,"Physical Class (Engine)"] = "Jet"
aircrafts_atl_df.at[18,"Engines"] = 3
aircrafts_atl_df.at[18,"Wingtip Configuration"] = "no winglets"

aircrafts_atl_df.at[19,"Physical Class (Engine)"] = "Jet"
aircrafts_atl_df.at[19,"Engines"] = 2
aircrafts_atl_df.at[19,"AAC"] = "C"
aircrafts_atl_df.at[19,"ADG"] = "III"
aircrafts_atl_df.at[19,"Approach Speed(Vref)"] = "138"
aircrafts_atl_df.at[19,"Wingtip Configuration"] = "winglets"
aircrafts_atl_df.at[19,"Wingspan, ft"] = '107,61'
aircrafts_atl_df.at[19,"Length, ft"] = '152,5591'
aircrafts_atl_df.at[19,"Tail Height, ft(@ OEW)"] = "29,53"
aircrafts_atl_df.at[19,"MTOW"] = "160 000"
aircrafts_atl_df.at[19,"Parking Area (WS x Length), sf"] = "16 417"


aircrafts_atl_df = aircrafts_atl_df.drop(labels=["TDG","Wheelbase, ft", "Cockpit to Main Gear (CMG)", "MGW(Outer to Outer)", "ATCT Weight Class", "TDG", "Main Gear Config", "Max RampMax Taxi"], axis="columns")

aircrafts_atl_df

,Model,Physical Class (Engine),Engines,AAC,ADG,Approach Speed(Vref),Wingtip Configuration,"Wingspan, ft","Length, ft","Tail Height, ft(@ OEW)",MTOW,ICAO Code,Wake Category,"Parking Area (WS x Length), sf"
0,ATR 42/72,Turboprop,2,B,III,114,no winglets,"88,75","89,13","25,08",49 603,AT72,M,7 910
1,Airbus A300,Jet,2,C,IV,137,wingtip fences,"147,14","177,43","54,67",363 760,A306,H,26 107
2,Airbus A319,Jet,2,C,III,126,wingtip fences,"111,88","111,02","39,73",168 653,A319,M,12 421
3,Airbus A320,Jet,2,C,III,136,wingtip fences,"111,88","123,27","39,63",171 961,A320,M,13 791
4,Airbus A321,Jet,2,C,III,140,wingtip fences,"111,88","146,03","39,70",206 132,A321,M,16 338
5,Airbus A330,Jet,2,C,V,137,winglets,"197,83","208,99","56,36",533 519,A333,H,41 344
6,Boeing 717,Jet,2,C,III,139,no winglets,"93,33","124,00","29,67",118 000,B712,M,11 573
7,Boeing 737,Jet,2,C,III,129,no winglets,"93,00","100,17","36,83",115 500,B732,M,9 316
8,Boeing 737 NG / Max,Jet,2,D,III,141,no winglets,"112,58","138,17","41,42",174 200,B739,M,15 555
9,Boeing 747,Jet,4,D,V,157,winglets,"195,67","231,85","64,25",610 000,B744,H,45 366


In [16]:
aircrafts_atl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Model                           20 non-null     object
 1   Physical Class (Engine)         20 non-null     object
 2   Engines                         20 non-null     object
 3   AAC                             20 non-null     object
 4   ADG                             20 non-null     object
 5   Approach Speed(Vref)            20 non-null     object
 6   Wingtip Configuration           20 non-null     object
 7   Wingspan, ft                    20 non-null     object
 8   Length, ft                      20 non-null     object
 9   Tail Height, ft(@ OEW)          20 non-null     object
 10  MTOW                            20 non-null     object
 11  ICAO Code                       20 non-null     object
 12  Wake Category                   20 non-null     obje

In [17]:
aircrafts_atl_df["Engines"] = aircrafts_atl_df["Engines"].astype(int)
aircrafts_atl_df["Approach Speed(Vref)"] = aircrafts_atl_df["Approach Speed(Vref)"].astype(int)
aircrafts_atl_df["Wingspan, ft"] = aircrafts_atl_df["Wingspan, ft"].apply(lambda x: float(x.replace(',','.')))
aircrafts_atl_df["Length, ft"] = aircrafts_atl_df["Length, ft"].apply(lambda x: float(x.replace(',','.')))
aircrafts_atl_df["Tail Height, ft(@ OEW)"] = aircrafts_atl_df["Tail Height, ft(@ OEW)"].apply(lambda x: float(x.replace(',','.')))
aircrafts_atl_df["MTOW"] = aircrafts_atl_df["MTOW"].apply(lambda x: int(x.replace(' ','')))
aircrafts_atl_df["Parking Area (WS x Length), sf"] = aircrafts_atl_df["Parking Area (WS x Length), sf"].apply(lambda x: int(x.replace(' ','')))
aircrafts_atl_df = aircrafts_atl_df.drop(labels=["AAC","ADG","Wake Category"], axis="columns" )
aircrafts_atl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Model                           20 non-null     object 
 1   Physical Class (Engine)         20 non-null     object 
 2   Engines                         20 non-null     int32  
 3   Approach Speed(Vref)            20 non-null     int32  
 4   Wingtip Configuration           20 non-null     object 
 5   Wingspan, ft                    20 non-null     float64
 6   Length, ft                      20 non-null     float64
 7   Tail Height, ft(@ OEW)          20 non-null     float64
 8   MTOW                            20 non-null     int64  
 9   ICAO Code                       20 non-null     object 
 10  Parking Area (WS x Length), sf  20 non-null     int64  
dtypes: float64(3), int32(2), int64(2), object(4)
memory usage: 1.7+ KB


In [18]:
aircrafts_atl_df

,Model,Physical Class (Engine),Engines,Approach Speed(Vref),Wingtip Configuration,"Wingspan, ft","Length, ft","Tail Height, ft(@ OEW)",MTOW,ICAO Code,"Parking Area (WS x Length), sf"
0,ATR 42/72,Turboprop,2,114,no winglets,88.75,89.1300,25.08,49603,AT72,7910
1,Airbus A300,Jet,2,137,wingtip fences,147.14,177.4300,54.67,363760,A306,26107
2,Airbus A319,Jet,2,126,wingtip fences,111.88,111.0200,39.73,168653,A319,12421
3,Airbus A320,Jet,2,136,wingtip fences,111.88,123.2700,39.63,171961,A320,13791
4,Airbus A321,Jet,2,140,wingtip fences,111.88,146.0300,39.70,206132,A321,16338
5,Airbus A330,Jet,2,137,winglets,197.83,208.9900,56.36,533519,A333,41344
6,Boeing 717,Jet,2,139,no winglets,93.33,124.0000,29.67,118000,B712,11573
7,Boeing 737,Jet,2,129,no winglets,93.00,100.1700,36.83,115500,B732,9316
8,Boeing 737 NG / Max,Jet,2,141,no winglets,112.58,138.1700,41.42,174200,B739,15555
9,Boeing 747,Jet,4,157,winglets,195.67,231.8500,64.25,610000,B744,45366


In [19]:
#def calc_bin_center(bins):
#    arr = []
#    for i in range(1,len(bins)):
#        arr.append((bins[i] + bins[i-1]) / 2)
#    return arr
#
#wingspan_adg_bin = [0,49,79,118,171,214,262]
#wingspan_adg_lbls = calc_bin_center(wingspan_adg_bin)
#tailheight_adg_bin= [0,20,30,45,60,66,80]
#tailheight_adg_lbls = calc_bin_center(tailheight_adg_bin)
#aac_bin = [0,90,120,140,165]
#aac_lbls = calc_bin_center(aac_bin)
#icao_bin = [0, 15500, 300000, 600000]
#icao_lbls = calc_bin_center(icao_bin)

In [20]:
#aircrafts_atl_df["Wingspan, ft"] = pd.cut(x=aircrafts_atl_df["Wingspan, ft"],
#                                          bins=wingspan_adg_bin,
#                                          labels=wingspan_adg_lbls)
#
#aircrafts_atl_df["Tail Height, ft(@ OEW)"] = pd.cut(x=aircrafts_atl_df["Tail Height, ft(@ OEW)"],
#                                          bins=tailheight_adg_bin,
#                                          labels=tailheight_adg_lbls)
#
#aircrafts_atl_df["Approach Speed(Vref)"] = pd.cut(x=aircrafts_atl_df["Approach Speed(Vref)"],
#                                                    bins=aac_bin,
#                                                    labels=aac_lbls)
#
#aircrafts_atl_df["MTOW"] = pd.cut(x=aircrafts_atl_df["MTOW"],
#                                                    bins=icao_bin,
#                                                    labels=icao_lbls)

In [21]:
aircrafts_atl_df.to_pickle(os.path.join(OUTPUT_FOLDER, "04_a_aircrafts.pkl"))

In [22]:
aircrafts_atl_df

,Model,Physical Class (Engine),Engines,Approach Speed(Vref),Wingtip Configuration,"Wingspan, ft","Length, ft","Tail Height, ft(@ OEW)",MTOW,ICAO Code,"Parking Area (WS x Length), sf"
0,ATR 42/72,Turboprop,2,114,no winglets,88.75,89.1300,25.08,49603,AT72,7910
1,Airbus A300,Jet,2,137,wingtip fences,147.14,177.4300,54.67,363760,A306,26107
2,Airbus A319,Jet,2,126,wingtip fences,111.88,111.0200,39.73,168653,A319,12421
3,Airbus A320,Jet,2,136,wingtip fences,111.88,123.2700,39.63,171961,A320,13791
4,Airbus A321,Jet,2,140,wingtip fences,111.88,146.0300,39.70,206132,A321,16338
5,Airbus A330,Jet,2,137,winglets,197.83,208.9900,56.36,533519,A333,41344
6,Boeing 717,Jet,2,139,no winglets,93.33,124.0000,29.67,118000,B712,11573
7,Boeing 737,Jet,2,129,no winglets,93.00,100.1700,36.83,115500,B732,9316
8,Boeing 737 NG / Max,Jet,2,141,no winglets,112.58,138.1700,41.42,174200,B739,15555
9,Boeing 747,Jet,4,157,winglets,195.67,231.8500,64.25,610000,B744,45366
